# Projeto 2 - Ciência dos Dados

Nome: Cicero Tiago Carneiro Valentim

Nome: Luiz Felipe Lazzaron

Nome: Marcos Vinícius da Silva

# Introdução

No presente trabalho, visa-se selecionar tweets relevantes para a classificação de postagens que possuem sinais de depressão por parte de seus autores. Nesse contexto, os integrantes dividiram os tweets em três categorias:

1. Irrelavantes ou Neutros. Tweets que não indicam quaisquer indício de depressão;
2. Relevantes. Tweets que possuem relevância na análise de indicação de depressão;
3. Possuem interesse no assunto. Tweets que não demonstram doença mental, mas indicam um interresse no tema e na prevenção de suicídio.

Desse modo, com tal classificação, pode-se desenvolver um marketing específico para os públicos 2 e 3; para os relevantes, pode-se oferecer auxílio e suporte psicológico; para aqueles que possuem interesse no tema "depressão", pode-se direcionar materiais que ensinem a como lidar com pessoas com depressão.

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from IPython.display import clear_output
from time import sleep
import script

___
## Autenticando no  Twitter

* Conta: ***@MarcosV96118169***

In [2]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @MarcosV96118169

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [3]:
#Produto escolhido:
produto = 'Depressão'

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [4]:
planilha_treinamento = pd.read_excel('depressao.xlsx', "Treinamento")
planilha_teste = pd.read_excel('depressao.xlsx', "Teste")
pd.set_option('display.max_colwidth', -1)

___
# Montando o Classificador Naive-Bayes


Primeiramente, iremos trabalhar com o dataframe treinamento, criando três banco de dados:
    
- **Grupo 1 - Irrelevantes/Neutros**: *group_1*
- **Grupo 2 - Relevantes**: *group_2*
- **Grupo 3 - Interesse no Assunto**: *group_3*


In [5]:
group_1 = planilha_treinamento[planilha_treinamento["Relevância"] == 1]

In [6]:
group_2 = planilha_treinamento[planilha_treinamento["Relevância"] == 2]

In [7]:
group_3 = planilha_treinamento[planilha_treinamento["Relevância"] == 3]

In [8]:
groups = [group_1,group_2,group_3]

In [9]:
str(list(groups[0]["Tweet Text"].values))

'[\'falei pra minha mãe sobre ter ansiedade e ela "n precisa ter ansiedade"\\n\\ndps eu disse q quando trabalhar irei na psicóloga e ela "vc ta com depressão?"\\n\\nfoda viu kkk\', \'e ainda vrm a depressao https://t.co/3qpgzs4ebu\', \'simplismente vejo hoje como a hipocrisia evoluiu, onde em setembro mil e um textos se ploriferam em redes socias, sendo que meses antes julgava como indigentes. depressão n é algo pra se discutir em x dia é toda hora. então para de postar pra aparecer e faça pela sua atitude\', \'tv:"tão fazendo campanha contra suicídio nas escolas já q este mal assim como a depressão atinge muitos adolescentes"\\nanimal: "dá uma vassoura pra trabalharem q n tem tempo pra pensarem em se matar"\', \'professora falou q sai da minha sala com depressão kkkkkkkkkkkk\', \'nessa escola depressao só existe em setembro\', \'a atriz e ex-aluna da puc-sp glamour garcia, participou hoje (10/09) do programa "encontro com fátima bernardes", onde falou sobre depressão enfrentada por el

# Limpeza e Tratamento dos Dados

In [10]:
#Bibliotecas para a Identificação de Emojis
!pip install emoji
from emoji import UNICODE_EMOJI
import emoji
import re 
import functools
import operator

### Função de Limpeza de um Tweet

In [11]:
# Recebe um texto normal e devolve uma lista de split
def space_cleaning(text):
    punctuation = '[!\-.:?;,''"@/]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    split_emoji = emoji.get_emoji_regexp().split(text_subbed)
    split_whitespace = [substr.split() for substr in split_emoji]
    split = functools.reduce(operator.concat,split_whitespace)
    return split

In [12]:
#Recebe um lista split e devolve uma lista split sem http ou https
def remove_http(lista_de_listas):
    lista = lista_de_listas
    for i in range(len(lista)):
        for j in range(len(lista[i])):
            if 'HTTP' in lista[i][j].capitalize():
                lista[i][j] = "http"
    
    for i in range(len(lista)):
        while lista[i].count("http") != 0:
            lista[i].remove("http")
    return lista

### Tratando os Dados da Base de Treinamento

In [13]:
def tratamento_do_grupo(grupo):
    total = []
    for tweet in grupo["Tweet Text"]:
        tweet_splited = space_cleaning(tweet)
       # tweet_splited = remove_http(tweet_splited)
        total.append(tweet_splited)
    return total

In [14]:
groups_tweet = [tratamento_do_grupo(groups[0]),tratamento_do_grupo(groups[1]),tratamento_do_grupo(groups[2])]

## Naive Bayes e Laplace Smoothing

In [15]:
#Número de Elementos De todos os grupos sem repetição
total_words = []
def lista_sem_repeticao(LISTA,lista):
    resposta = LISTA
    for tweet in lista:
        for palavra in tweet:
            if palavra not in resposta:
                resposta.append(palavra)
    return resposta
lista_sem_repeticao(total_words,groups_tweet[0])
lista_sem_repeticao(total_words,groups_tweet[1])
lista_sem_repeticao(total_words,groups_tweet[2])
N =len(total_words)

In [16]:
N

2970

In [17]:
# Algoritmo de Laplace
def laplace(num_vezes_no_grupo, num_elementos_do_grupo, num_elementos_sem_repeticao):
    probabilidade = (num_vezes_no_grupo + 1 ) / ( num_elementos_do_grupo + num_elementos_sem_repeticao)
    return probabilidade

In [18]:
#Número de vezes que a palavra aparece no grupo
def numero_de_vezes(palavra_analisada,grupo):
    n = 0
    for tweet in grupo:
        n += tweet.count(palavra_analisada)
#         for palavra in tweet:
#             if palavra ==  palavra_analisada:
#                 n +=1
    return n

In [19]:
#Calculando a probabilidade para cada palavra
resultados = []
for group in groups_tweet:
    dictionary= {}
    for tweet in group:
        for word in tweet:
            probabilidade = laplace(numero_de_vezes(word,group),len(group),N)
            dictionary[word] = probabilidade
    resultado = pd.Series(dictionary)
    resultados.append(resultado)

In [20]:
#Número de Palavras em Cada uma dos Grupos
def conta_palavras(grupo):
    n = 0
    for tweet in grupo:
        n += len(tweet)
    return n

In [21]:
N_1 = conta_palavras(groups_tweet[0])
N_2 = conta_palavras(groups_tweet[1])
N_3 = conta_palavras(groups_tweet[2])

___
## Verificando a performance na Planilha Testes

Agora você deve testar o seu classificador com a base de Testes.

In [22]:
teste = planilha_teste["Tweet Text"]

In [23]:
def limpeza_tweet(tweet):
    tweet_limpo = space_cleaning(str(tweet))
    return tweet_limpo

In [24]:
teste_tweet = []
for tweet in teste:
    teste_tweet.append(limpeza_tweet(tweet))

In [25]:
serie_teste = pd.Series(teste_tweet)

## Função de Análise

Como os resultados estão muito pequenos, usou-se a função do logarítmo neperiano, ln(x), para os valores ficarem com números mais adequados para o Python.

In [26]:
#Função de Análise de Frase
def analisa_frase(frase):
    condicional_grupo1 = 1
    condicional_grupo2 = 1
    condicional_grupo3 = 1
    lista = [condicional_grupo1, condicional_grupo2,condicional_grupo3]
    
    for palavra in frase:
        if (palavra not in resultados[0].index):
            condicional_grupo1 *= laplace(0,N_1,N)
        else: 
            condicional_grupo1 *= resultados[0][palavra]
            
    for palavra in frase:
        if (palavra not in resultados[1].index):
            condicional_grupo2 *= laplace(0,N_1,N) 
        else:
            condicional_grupo2 *= resultados[1][palavra]
            
    for palavra in frase:
        if (palavra not in resultados[2].index):
            condicional_grupo3 *= laplace(0,N_1,N) 
        else:
            condicional_grupo3 *= resultados[2][palavra]
            
    return [condicional_grupo1,condicional_grupo2,condicional_grupo3]

In [27]:
analisa_frase("Gostei muito de você")

[1.1116983151648638e-60, 4.458025834425154e-61, 1.3518821708300341e-62]

In [28]:
def identifica_valor_maximo(lista_de_resultados):
    if lista_de_resultados[0] == max(lista_de_resultados):
        return 1
    elif lista_de_resultados[1] == max(lista_de_resultados):
        return 2
    else:
        return 3

In [29]:
def resultado(database_coluna, n):
    return identifica_valor_maximo(analisa_frase(limpeza_tweet(database_coluna[n])))

In [30]:
estimativa = []
for k in range(0,len(teste)):
    estimativa.append(resultado(teste,k))

In [31]:
planilha_teste["Relevância_Estimada"] = estimativa

In [32]:
Acertos = 0
for n in planilha_teste.index:
    if planilha_teste["Relevância_Prevista"][n] == planilha_teste["Relevância_Estimada"][n]:
        Acertos += 1

In [33]:
planilha_treinamento["Relevância"].value_counts(normalize = True)

1.0    0.492205
2.0    0.285078
3.0    0.222717
Name: Relevância, dtype: float64

In [34]:
planilha_teste["Relevância_Prevista"].value_counts(normalize = True)

1    0.540000
3    0.236667
2    0.223333
Name: Relevância_Prevista, dtype: float64

In [35]:
planilha_teste["Relevância_Estimada"].value_counts(True)

1    0.93
2    0.05
3    0.02
Name: Relevância_Estimada, dtype: float64

In [36]:
Acertos/len(planilha_teste)

0.55

# Planilha DataBase.xlsx

In [37]:
DB = pd.read_excel("DataBase.xlsx")
db_tweet = []
for tweet in DB:
    db_tweet.append(limpeza_tweet(tweet))
serie_db = pd.Series(db_tweet)
resultado(DB["Tweet Text"], n)
estimativa = []
for k in range(0,len(DB["Tweet Text"])):
    estimativa.append(resultado(DB["Tweet Text"],k))
DB["Relevância_Estimada"] = estimativa

In [38]:
DB["Relevância_Estimada"].value_counts()

1    2463
2    121 
3    49  
Name: Relevância_Estimada, dtype: int64

In [39]:
DB[["Relevância_Estimada","Tweet Text"]]

,Relevância_Estimada,Tweet Text
0,2,"@damadeferroofic depressão é uma doença dama, muito intimo, que não impede de termos brio, respeito, simpatia, educação..\nquando eu caio eu fico sozinha, pois não quero expor as pessoas..\nmas graças a deus, não tive surtos..\nmas, o twitter ajudou mesmo..\n😉😉😉🤔🤔🤔"
1,1,"quem tem fé também adoece, fica triste, fica ansioso, tem depressão e sofre. a fé não livra ninguém de passar por problemas, mas ela ajuda a pessoa a não ficar encostada na escuridão por muito tempo. 🍃"
2,1,"já dizia freud “antes de diagnosticar a si mesmo com depressão ou baixa autoestima, primeiro tenha certeza de que você não está, de fato, cercado por idiotas.”"
3,1,gente uma criança de 10 anos cometeu suicídio e ainda tem gente que acha qhe depressão é frescura e brincadeira ? 😔😔😢😢😭😭💔💔 https://t.co/qwkoqxj4ew
4,1,"""vou fingir que tenho depressão, assim todos se solidarizam"". pera... isso já flopou\n""vou fingir que estou sendo ameaçado"". #forçafelipe"
5,1,spunkytristinho pq nariz não funciona pqp que depressão
6,1,praça e uma senhora tava comentando que a filha da vizinha tentou suicídio e a ouvinte retrucou falando que a jovenzinha não foi bem educada e que agora quer entrar na moda dos jovens de terem depressão. só que ++++
7,1,"amei a piadinha com doença psicológica alheia, se fosse depressão geral: cancela essa menina https://t.co/kvhx12x1s3"
8,1,n cnsg dormir vsfd odeio a depressão\n\n#setembroamarelo
9,2,ansiedade = não conseguir dormir de noite.\ndepressão= não conseguir acordar de manhã.\n😭😭😭


In [40]:
def remove_http(dataframe):
    """
    usar dessa maneira:
    DB = remove_http(DB)
    """
    data_aux = dataframe
    lista_tweets = list(dataframe.loc[:,"Tweet Text"].values)
    lista_reserva = []
    for i in range(len(lista_tweets)):
        lista_tweets[i] = lista_tweets[i].split()
        for t in range(len(lista_tweets[i])):
            if "http" in lista_tweets[i][t]:
                lista_tweets[i][t] = "http"
            if "@" in lista_tweets[i][t]:
                lista_tweets[i][t] = "@"
        while lista_tweets[i].count("http") != 0:
            lista_tweets[i].remove("http")
        while lista_tweets[i].count("@") != 0:
            lista_tweets[i].remove("@")
        lista_tweets[i] = ' '.join(lista_tweets[i])
    data_aux.loc[:,"Tweet Text"] = lista_tweets
    return data_aux

In [41]:
DB = remove_http(DB)
print(DB)

      Favorite Count           Phone Type  Relevância  Retweets  \
0     0               Twitter for Android NaN          0          
1     0               Twitter for Android NaN          0          
2     0               Twitter for iPhone  NaN          0          
3     0               Twitter for iPhone  NaN          0          
4     0               Twitter Web App     NaN          0          
5     0               Twitter for Android NaN          0          
6     0               Twitter for iPhone  NaN          0          
7     0               Twitter for Android NaN          0          
8     0               Twitter for Android NaN          0          
9     1               Twitter for iPhone  NaN          0          
10    2               Twitter for iPhone  NaN          0          
11    0               Twitter for Android NaN          0          
12    17              Twitter for iPhone  NaN          3          
13    1               Twitter Web App     NaN          0      

___
### Concluindo

In [42]:
def return_tweet(quantidade, autorizacao):
    """return_tweet(autorizacao, quantidade)"""
    api = tweepy.API(autorizacao)
    i = 0
    __lista__ = []
    anterior = {'User Name': 'foo',
                'Tweet Created At': 'foo','Tweet Text': 'foo','Relevância': 'foo',
                'User Location': 'foo','Phone Type': 'foo',
                'Favorite Count': 'foo','Retweets':'foo'}
    while i < quantidade:
        for msg in tweepy.Cursor(api.search, q="{0} -filter:retweets".format("depressao"), lang='pt', tweet_mode="extended").items():
            new_msg = {'Tweet Text': msg.full_text.lower()}
            i += 1
            if  new_msg["Tweet Text"] not in __lista__:
                __lista__.append(new_msg)
                if resultado(list(new_msg["Tweet Text"]),0) == 1:
                    __resultado__ = "Irrelevante"
                elif resultado(list(new_msg["Tweet Text"]),0) == 2:
                    __resultado__ = "Preocupante"
                elif resultado(list(new_msg["Tweet Text"]),0) == 3:
                    __resultado__ = "Demonstra interesse pelo tema"
                print("[{0}]\nClassificação:\t{1}\nTexto:\t{2}".format(i,__resultado__, new_msg["Tweet Text"]))
            print("\t -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-\n\n")
            sleep(4.5)
            if i >= quantidade:
                break
        print("[Fim do Programa]")

In [43]:
return_tweet(50,auth) # return_tweet(quantidade, autenticação)

[1]
Classificação:	Irrelevante
Texto:	gente ajudem por favor, há uma família desesperada!! 

quem encontrar ele entre em contato!! 

vamos ajudar, oque indica e que ele estava c depressao e derrepente sumiu !!😣 https://t.co/1jixiu1btr
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[2]
Classificação:	Irrelevante
Texto:	governo anuncia campanha para combater a depressão e o suicídio com foco nos jovens https://t.co/rchgnbzi85 https://t.co/4qqqgjuzro
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[3]
Classificação:	Irrelevante
Texto:	eu devia mesmo é parar de deixar a depressão me atrasar e conseguir dar uma guinada na minha vida, porque ficar perto da minha mãe só me coloca mais pra baixo
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[4]
Classificação:	Demonstra interesse pelo tema
Texto:	sabe de uma coisa? durante todos esses anos me mantive calado sobre um assunto que é muito sério, e deveria ser levado a sério.

a porra do setembro. eu vejo muita gente por ai falando sobre a

[37]
Classificação:	Irrelevante
Texto:	foi só falar de depressão e suicídio na aula que eu já comecei chorar só de pensar na minha mãe😭
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[38]
Classificação:	Preocupante
Texto:	regra de três  resolve até  depressão. https://t.co/jvjbrc5fdf
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[39]
Classificação:	Demonstra interesse pelo tema
Texto:	depressão ta acabando cmg aos poucos, minha coroa e eu nessa situação ta me destruindo, tem que ser forte demais pra se manter de pé..
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[40]
Classificação:	Irrelevante
Texto:	eu: mãe, to com depressão 

minha mãe: é falta de deus, vai orar
	 -=-=-=-=-= aguardando 4.5 segundos =-=-=-=-=-


[41]
Classificação:	Irrelevante
Texto:	a pessoa da essas birra minha personalidade oculta que é psicóloga com 16 anos de experiência rapidamente se manifesta e diz: provável carência de atenção dos pais durante a infância e adolescência além de complexo de inferiori

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**